# Bank Customer Churn

The customer is the credit department of the bank. It is necessary to find out whether the marital status and the number of children of the client affect the fact of repaying the loan on time. Input data from the bank - statistics on the solvency of customers.
The results of the study will be taken into account when building a credit scoring model - a special system that assesses the ability of a potential borrower to return a loan to a bank.

**The purpose of the study** is to test four hypotheses:

1. Is there a relationship between the number of children and loan repayment on time
2. Is there a relationship between marital status and loan repayment on time
3. Is there a relationship between income level and loan repayment on time
4. How different purposes of the loan affect its repayment on time

**Research Plan**

The study will take place in four stages:

* Overview of data.
* Data preprocessing.
* Data categorization.
* Hypothesis testing.

## Data review

In [1]:
# pandas import and file read
import pandas as pd
import numpy as np
df = pd.read_csv('/datasets/data.csv')

In [2]:
df.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


DataFrame has twelve columns.

According to the documentation:
    
* `children` — number of children
* `days_employed` — work experience (in days)
* `dob_years` — client age (in years)
* `education` — level of education
* `education_id` — education level id
* `family_status` — family status
* `family_status_id` — family status id
* `gender` — gender
* `income_type` — income type
* `debt` — had overdue payment
* `total_income` — monthly income
* `purpose` — purpose of debt

In [4]:
df.isna().agg(['sum', 'mean'])

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
sum,0.0,2174.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2174.000000,0.0
mean,0.0,0.100999,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.100999,0.0


Columns `days_employed` and `total_income` have 2174 or 10% of missed values.

These values should be filled with medians because these columns may have a high level of variance among categories (e.g. 'unemployed' vs 'retired').

In [5]:
age = df['dob_years'].unique()
print(sorted(age))

[0, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75]


In [6]:
df[df['dob_years'] == 0].head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
99,0,346541.618895,0,Среднее,1,женат / замужем,0,F,пенсионер,0,71291.522491,автомобиль
149,0,-2664.273168,0,среднее,1,в разводе,3,F,сотрудник,0,70176.435951,операции с жильем
270,3,-1872.663186,0,среднее,1,женат / замужем,0,F,сотрудник,0,102166.458894,ремонт жилью
578,0,397856.565013,0,среднее,1,женат / замужем,0,F,пенсионер,0,97620.687042,строительство собственной недвижимости
1040,0,-1158.029561,0,высшее,0,в разводе,3,F,компаньон,0,303994.134987,свой автомобиль
1149,0,-934.654854,0,среднее,1,женат / замужем,0,F,компаньон,0,201852.430096,покупка недвижимости
1175,0,370879.508002,0,среднее,1,женат / замужем,0,F,пенсионер,0,313949.845188,получение дополнительного образования
1386,0,-5043.219890,0,высшее,0,женат / замужем,0,M,госслужащий,0,240523.618071,сделка с автомобилем
1890,0,NaN,0,высшее,0,Не женат / не замужем,4,F,сотрудник,0,NaN,жилье
1898,0,370144.537021,0,среднее,1,вдовец / вдова,2,F,пенсионер,0,127400.268338,на покупку автомобиля


In [7]:
df['gender'].value_counts()

F      14236
M       7288
XNA        1
Name: gender, dtype: int64

In [8]:
df['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

In [9]:
df['days_employed'].describe()

count     19351.000000
mean      63046.497661
std      140827.311974
min      -18388.949901
25%       -2747.423625
50%       -1203.369529
75%        -291.095954
max      401755.400475
Name: days_employed, dtype: float64

Aside from missed values, the dataframe also has anomalies:

1. Column `days_employed` contains negative values and extremely large values.
2. Column `dob_years` contains zero values.
3. Column `children` has values -1 and 20.
4. Column `gender` contains value `XNA`.

**Conclusion**

Each row of the table contains information about bank clients. Some of the columns describe socio-demographic information: age, education, gender, marital status, and the number of children. The rest of the data tells about economic indicators: seniority, type of employment, income, whether he had debts to repay loans, and the purpose of obtaining a loan.

It can be assumed that we have enough data to test hypotheses. But to move forward we need to handle missed values, duplicates, and anomalies.

## Data preparation

### Handling missing values

Let's fill in missed values in columns `days_employed` and `total_income` with median values based on income type.

In [10]:
#Create dictionary with pairs(income type : median value)
income_type_income_median = df.groupby('income_type').median()['total_income'].to_dict()    
income_type_employed_median = df.groupby('income_type').median()['days_employed'].to_dict()    

In [11]:
income_type_income_median

{'безработный': 131339.7516762103,
 'в декрете': 53829.13072905995,
 'госслужащий': 150447.9352830068,
 'компаньон': 172357.95096577113,
 'пенсионер': 118514.48641164352,
 'предприниматель': 499163.1449470857,
 'сотрудник': 142594.39684740017,
 'студент': 98201.62531401133}

In [12]:
#Make a function that fills missed values with medians based on income type from dictionary and apply it to the data
def total_income_fill_median(row):
    total_income = row['total_income']
    income_type = row['income_type']
    if np.isnan(row['total_income']):      
        return income_type_income_median[income_type]
    return total_income
    
df['total_income'] = df.apply(total_income_fill_median, axis=1)

In [13]:
def days_employed_fill_median(row):
    days_employed = row['days_employed']
    income_type = row['income_type']
    if np.isnan(row['days_employed']):      
        return income_type_employed_median[income_type]
    return days_employed
       
df['days_employed'] = df.apply(days_employed_fill_median, axis=1)

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     21525 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      21525 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


### Handling anomalies

In [15]:
df['days_employed'] = df['days_employed'].abs()
df['children'] = df['children'].abs()
df['children'] = df['children'].replace(20, 2)
df = df.loc[df['gender'] != 'XNA']
df = df.loc[df['dob_years'] != 0]

In [16]:
df['gender'].value_counts()

F    14164
M     7259
Name: gender, dtype: int64

First, let's get rid of the negative values in the `days_employed` and `children` columns by converting them to absolute values. Negative values in the `days_employed` column are found only among people who still work. It seems that negative values separate currently employed from unemployed and retired. There are also abnormally large values in the `days_employed` column. Since this column is not used in calculations, we will leave these values unchanged.

Let's also assume that the values '20' in the children column are due to human error (accidentally adding a zero after the 2 because the value is very out of the 0-5 range) and replace them with the value 2.

The '0' values in the `dob_years` column will have to be removed as they occur across different customer categories such as: 'employee', 'companion', and 'retired'. In this case, filling with the mean or median will not work.

### Data type change

In [17]:
df['total_income'] = df['total_income'].astype('int')

In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21423 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21423 non-null  int64  
 1   days_employed     21423 non-null  float64
 2   dob_years         21423 non-null  int64  
 3   education         21423 non-null  object 
 4   education_id      21423 non-null  int64  
 5   family_status     21423 non-null  object 
 6   family_status_id  21423 non-null  int64  
 7   gender            21423 non-null  object 
 8   income_type       21423 non-null  object 
 9   debt              21423 non-null  int64  
 10  total_income      21423 non-null  int64  
 11  purpose           21423 non-null  object 
dtypes: float64(1), int64(6), object(5)
memory usage: 2.1+ MB


### Duplicates handling

Let's check for duplicates and remove them.

In [19]:
df.duplicated().sum()

54

In [20]:
df = df.drop_duplicates().reset_index(drop=True)

In [21]:
df.duplicated().sum()

0

Let's check the data for implicit duplicates

In [22]:
display(df['education'].sort_values().unique())

array(['ВЫСШЕЕ', 'Высшее', 'НАЧАЛЬНОЕ', 'НЕОКОНЧЕННОЕ ВЫСШЕЕ',
       'Начальное', 'Неоконченное высшее', 'СРЕДНЕЕ', 'Среднее',
       'УЧЕНАЯ СТЕПЕНЬ', 'Ученая степень', 'высшее', 'начальное',
       'неоконченное высшее', 'среднее', 'ученая степень'], dtype=object)

Column `education` contains similar values but written in lower/upper cases. Let's bring the data to lowercase, use snake case and remove duplicates.

In [23]:
df['education'] = df['education'].str.lower()
df['family_status'] = df['family_status'].str.lower()

In [24]:
df.duplicated().sum()

17

In [25]:
df = df.drop_duplicates().reset_index(drop=True)

In [26]:
df.duplicated().sum()

0

### Creating dictionaries and decomposition of the original dataframe

In [27]:
#создание словарей для education и family_status
education_dict = df[['education_id','education']]
family_dict = df[['family_status_id', 'family_status']]
education_dict = education_dict.drop_duplicates().reset_index(drop=True)
family_dict = family_dict.drop_duplicates().reset_index(drop=True) 
df.drop(columns = ['education', 'family_status'], axis = 1) 

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,0,0,F,сотрудник,0,253875,покупка жилья
1,1,4024.803754,36,1,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623.422610,33,1,0,M,сотрудник,0,145885,покупка жилья
3,3,4124.747207,32,1,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266.072047,53,1,1,F,пенсионер,0,158616,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...
21347,1,4529.316663,43,1,1,F,компаньон,0,224791,операции с жильем
21348,0,343937.404131,67,1,0,F,пенсионер,0,155999,сделка с автомобилем
21349,1,2113.346888,38,1,1,M,сотрудник,1,89672,недвижимость
21350,3,3112.481705,38,1,0,M,сотрудник,1,244093,на покупку своего автомобиля


### Income categories

Let's categorize clients by income into the following categories:

* 0–30000 — 'E';
* 30001–50000 — 'D';
* 50001–200000 — 'C';
* 200001–1000000 — 'B';
* 1000001 и выше — 'A'.

In [28]:
def category(income):
    if income <= 30000:
        return 'E'
    if income <= 50000:
        return 'D'
    if income <= 200000:
        return 'C'
    if income <= 1000000:
        return 'B'
    return 'A'

df['total_income_category'] = df['total_income'].apply(category)

In [29]:
df['total_income_category'].value_counts()

C    15942
B     5015
D      348
A       25
E       22
Name: total_income_category, dtype: int64

### Loan purpose categories

Let's bring the loan `purpose` column to a single view by dividing the values into the following categories:

* `операции с автомобилем` ('car loan')
* `операции с недвижимостью` ('mortgage')
* `проведение свадьбы` ('wedding loan')
* `получение образования` ('education loan')

In [30]:
def purpose_category(purpose):
    if 'автомобил' in purpose:
        return 'операции с автомобилем'
    elif 'недвиж' in purpose or 'жил' in purpose:
        return 'операции с недвижимостью'
    elif 'свадьб' in purpose:
        return 'проведение свадьбы'
    else:
        return 'получение образования'
    
df['purpose_category'] = df['purpose'].apply(purpose_category)

In [31]:
df['purpose_category'].value_counts()

операции с недвижимостью    10763
операции с автомобилем       4284
получение образования        3995
проведение свадьбы           2310
Name: purpose_category, dtype: int64

### Number of children 

Let's divide the data in the `children` column into the following categories: 

* 0 - 'бездетные' ('childless')
* 1-2 - '1-2 детей' ('1-2 children')
* 3 и более - 'многодетные' ('having many children')

In [32]:
def category(children):
    if children == 0:
        return 'бездетные'
    if children <= 2:
        return '1-2 детей'
    return 'многодетные'

df['children_category'] = df['children'].apply(category)

In [33]:
df['children_category'].value_counts()

бездетные      14021
1-2 детей       6953
многодетные      378
Name: children_category, dtype: int64

### Questions - Answers

#### Question 1:

Let's check if there is a relation between the number of children and returning a loan in time.

In [34]:
df.groupby('children_category')['debt'].agg(['count','mean'])

,count,mean
children_category,,
1-2 детей,6953,0.092622
бездетные,14021,0.075458
многодетные,378,0.082011


In [35]:
df.pivot_table(index=['children_category'],columns=['gender'],values='debt',aggfunc=['count', 'mean'])

count            mean          
gender                F     M         F         M
children_category                                
1-2 детей          4402  2551  0.086779  0.102705
бездетные          9470  4551  0.062196  0.103054
многодетные         230   148  0.078261  0.087838

#### Answer 1:

We can make the following conclusions from the obtained results:

1. Most often, the loan is repaid on time by clients (7.5% of non-repayments on time) who do not have children.
2. The largest share of loan defaults - 9.3%, among clients with 1-2 children.
3. Among those with many children, the share of clients who do not repay the loan on time is 8.2%

Also, if we include gender, we can see that men default more often in each category.

#### Question 2:

Let's check if there is any relation between marital status and default.

In [36]:
df.groupby('family_status')['debt'].agg(['count','mean'])

,count,mean
family_status,,
в разводе,1185,0.071730
вдовец / вдова,954,0.064990
гражданский брак,4129,0.093485
женат / замужем,12290,0.075427
не женат / не замужем,2794,0.097709


In [37]:
df.pivot_table(index=['family_status'],columns=['gender'],values='debt',aggfunc=['count', 'mean'])

count            mean          
gender                    F     M         F         M
family_status                                        
в разводе               927   258  0.065804  0.093023
вдовец / вдова          899    55  0.056730  0.200000
гражданский брак       2829  1300  0.082008  0.118462
женат / замужем        7726  4564  0.068211  0.087642
не женат / не замужем  1721  1073  0.068565  0.144455

#### Answer 2:

If we look at the relationship between marital status and loan repayment on time, we can draw the following conclusions:

1. The share of clients who do not repay loans on time is the highest among the categories 'single' and 'civil marriage' - 9.8% and 9.4% respectively.
2. The smallest share of clients - 6.5%, who do not repay their loans on time is among the 'widower/widow' category. Also, 'divorced' and 'married' clients are more responsible with only 7.2% and 7.5% of late payments respectively.

Also, if we add the division by gender, we can see that men default more often in each category.

#### Question 3:

Let's check if there is a relation between income and paying on time.

In [38]:
df.groupby('total_income_category')['debt'].agg(['count','mean'])

,count,mean
total_income_category,,
A,25,0.080000
B,5015,0.070788
C,15942,0.084870
D,348,0.060345
E,22,0.090909


In [39]:
df.pivot_table(index=['total_income_category'],columns='gender',values='debt',aggfunc=['count', 'mean'])

count            mean          
gender                     F     M         F         M
total_income_category                                 
A                         10    15  0.000000  0.133333
B                       2697  2318  0.060808  0.082399
C                      11072  4870  0.072977  0.111910
D                        307    41  0.048860  0.146341
E                         16     6  0.125000  0.000000

#### Answer 3:

We can see from the data that there is no linear relation between income and paying in time.

The highest default rate - 9.1% is within the low income category (below 30000).

They are followed by clients with income between 50,000 and 200,000 - default rate is 8.5%.

The clients with incomes 30000 to 50000 have the lowest default rate.

Also, clients with income above 1000000 have a default rate of 8%.

If we add the division by gender, we can notice an interesting thing. Men are more likely to fail to repay loans on time in all categories except for one - clients with incomes below 30,000. On the other hand, this phenomenon may be present due to a lack of data on the category. In our case, 22 rows of data is not a representative sample. Additional data are needed to confirm or reject this hypothesis.

#### Question 4:

Let's check how paying on time depends on the loan purpose.

In [40]:
df.groupby('purpose_category')['debt'].agg(['count','mean'])

,count,mean
purpose_category,,
операции с автомобилем,4284,0.093371
операции с недвижимостью,10763,0.072378
получение образования,3995,0.092616
проведение свадьбы,2310,0.079654


In [41]:
df.pivot_table(index=['purpose_category'],columns='gender',values='debt',aggfunc=['count', 'mean'])

count            mean          
gender                       F     M         F         M
purpose_category                                        
операции с автомобилем    2842  1442  0.081633  0.116505
операции с недвижимостью  7056  3707  0.062783  0.090639
получение образования     2652  1343  0.078054  0.121370
проведение свадьбы        1552   758  0.068943  0.101583

#### Answer 4:

The highest loan default rate is within clients, who get a car loan - 9.4% and a student loan - 9.2%.

The lowest loan default rate is within clients with mortgages - 7.2%.

In the middle are the clients who get a wedding loan with 8% of defaults.

If we add the division by gender, we can notice that men are more likely to fail to repay loans on time in all categories.

## Conclusion:

In the course of the project, we found missing values and anomalies in the data. We replaced missed values in `days_employed` and `total_income` with medians by income type. We handled the anomalies in the `dob_years`, `children`, and `gender` columns. Then we changed the data type in the `total_income` column to 'int'. After that, we removed the explicit and implicit duplicates and added income categories, the purpose of the loan, and the number of children categories.

After the analysis we obtained the following results:

1. Clients without children are more likely to repay the loan on time. The share of late payments is lower among families with many children than among families with 1-2 children.

2. Unmarried and married clients have more debts than other categories. Widows/widowers are more likely to repay loans on time.

3. Contrary to expectations, the results show that there is no clear linear relationship between the level of income and the repayment of the loan on time. While, as expected, the lowest-income clients are more likely to default, the highest-income clients are also quite likely to default on loans.

4. Real estate loans are more likely to be returned on time than car and education loans.

Also, in the course of the study, we found a relation between gender and loan repayment on time. Thus, men are more likely to delay a loan payment than women.